In [ ]:
!pip install boto3 langchain -Uq
!pip install --upgrade pip
!pip install langchain_community
!pip install -U langchain-aws


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.9/378.9 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.2 MB/s eta 0:00:00
ERROR: Operation cancelled by user
  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-package

In [ ]:
import langchain
import langchain_community
#import langchain_aws
#print(langchain.__version__)
#print(langchain_community.__version__)

In [ ]:
import boto3
import json
import socket
import pandas as pd

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import BedrockChat
from langchain.prompts import PromptTemplate # Import PromptTemplate
from langchain.llms import Bedrock # Import the Bedrock class from langchain.llms


In [ ]:
def setSettings(access_key,secret_key,region):
  AWS_ACCESS_KEY = access_key
  AWS_SECRET_KEY = secret_key
  REGION = region

  session = boto3.Session(
                          aws_access_key_id=AWS_ACCESS_KEY,
                          aws_secret_access_key=AWS_SECRET_KEY,
                          region_name=REGION
                          )

  bedrock_runtime = session.client('bedrock-runtime')

  return bedrock_runtime


In [ ]:
bedrock_runtime= setSettings("your access key", "your secret access key",'your region')

In [ ]:
def getPredictedCategory(model,bedrock,final_prompt):
    # Initilization
    instruction="You are a scientific researcher that needs to perform the following task."
    # Model setting
    if model == "llama2-70b":
        model_id = "meta.llama2-70b-chat-v1"
        model_kwargs = {"max_gen_len":2048, "temperature":0.1, "top_p":1}
        instruction="<s>[INST] <<SYS>>\n"+instruction+"\n<</SYS>>\n"
        final_prompt = final_prompt+" [/INST]"
    elif model == "llama3-70b":
       model_id = "meta.llama3-70b-instruct-v1:0"
       model_kwargs = {"max_gen_len":2048, "temperature":0.1, "top_p":1}
       instruction="<s>[INST] <<SYS>>\n"+instruction+"\n<</SYS>>\n"
       final_prompt = final_prompt+" [/INST]"
    elif model == "llama3-8b":
       model_id = "meta.llama3-8b-instruct-v1:0"
       model_kwargs = {"max_gen_len":2048, "temperature":0.1, "top_p":1}
       #instruction="<s>[INST] <<SYS>>\n"+instruction+"\n<</SYS>>\n"
       #final_prompt = final_prompt+" [/INST]"
    elif model == "mistral-7b-instruct":
       model_id = "mistral.mistral-7b-instruct-v0:2"
       model_kwargs = {"max_tokens":2048, "temperature":0.1, "top_p":1, "top_k":100}
       final_prompt = "<s>[INST] "+final_prompt+" [/INST]"
    elif model == "mistral-8x7b-instruct":
       model_id = "mistral.mixtral-8x7b-instruct-v0:1"
       model_kwargs = {"max_tokens":2048, "temperature":0.1, "top_p":1, "top_k":100}
       final_prompt = "<s>[INST] "+final_prompt+" [/INST]"
    elif model == "mistral-large":
       model_id = "mistral.mistral-large-2402-v1:0"
       model_kwargs = {"max_tokens":2048, "temperature":0.1, "top_p":1, "top_k":100}
       final_prompt = "<s>[INST] "+final_prompt+" [/INST]"
    elif model == "cohere":
       model_id = "cohere.command-text-v14"
       model_kwargs = {"max_tokens":2048, "temperature":0.1, "p":1, "k":100}
    elif model == "cohere-light":
       model_id = "cohere.command-light-text-v14"
       model_kwargs = {"max_tokens":2048, "temperature":0.1, "p":1, "k":100}
    elif model == "claude-haiku":
         model_id = "anthropic.claude-3-haiku-20240307-v1:0"
         model_kwargs =  {"max_tokens": 2048, "temperature": 0.1,"top_k": 100, "top_p": 1,   "stop_sequences": ["\n\nHuman"], }
    elif model == "claude-sonnet":
         model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
         model_kwargs =  {"max_tokens": 2048, "temperature": 0.1,"top_k": 100, "top_p": 1,   "stop_sequences": ["\n\nHuman"], }
    else:
            raise Exception("Error with the MODEL parameter. Valid values are llama, mistral , sonnet or cohere")

    # Testing: Delete later.
    #print("Id:"+model_id)
    #print("Prompt:"+final_prompt)

    # Model Invoke
    model = BedrockChat(
                        client=bedrock,
                        model_id=model_id,
                        model_kwargs=model_kwargs,
                        )
    # Invoke Example
    messages = [
                  ("system", instruction),
                  ("human", "{question}"),
                ]

    prompt = ChatPromptTemplate.from_messages(messages)
    chain = prompt | model | StrOutputParser()

      # Chain Invoke
    response = chain.invoke({"question": final_prompt})

    return response


In [ ]:
#df = pd.read_excel("LR_Annotation_Updated_Cleaned_Final_03.xlsx", "SentenceExamplesAll")
#df = pd.read_excel("LR_Annotation_Updated_Cleaned_Final_03.xlsx", "SentenceExamples28")
#df = pd.read_excel("LR_Annotation_Updated_Cleaned_Final_03.xlsx", "Sentences700")
df = pd.read_excel("Raters_All.xlsx")
#df = pd.read_excel("LR_Annotation_Updated_Cleaned_Final_03.xlsx", "SentenceExamples140")

df

,Id,Sentence,Fco,Pablo,Paula,Vote,Unique,Unique_Vote
0,sen_219,Although there is a long tradition of conducti...,RESEARCH GAP,RESULT,RESULT,RESULT,NO,NO
1,sen_62,Recent multicenter randomized controlled trial...,RESULT,LIMITATION,LIMITATION,LIMITATION,NO,NO
2,RecSys 02 DES 4,in our recent work on music genre exploration ...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES
3,RecSys 04 DES 3,in order to reflect the users complete spectru...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES
4,RecSys 05 DES 1,mcauley et al. [21] mine key attributes from t...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES
...,...,...,...,...,...,...,...,...
135,ISWC 2022 04 DES 2,tcomplex [21] is a temporal extension of compl...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES
136,ISWC 2022 04 DES 7,tie [41] is a time-aware incremental embedding...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES
137,ISWC 2022 05 DES 15,a neural topic model is employed in [28] to ex...,DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES
138,NEURIPS02 DES 10,"also, shin et al. (2022) extended the classifi...",DESCRIPTION,DESCRIPTION,DESCRIPTION,DESCRIPTION,YES,YES


In [ ]:
# Function to get 20 random rows from each category
def get_sample(df, column, n):
    return df.groupby(column).apply(lambda x: x.sample(n=min(len(x), n))).reset_index(drop=True)

# Get 20 random rows from each category
#df = get_sample(df, 'Category', 100)
df = get_sample(df, 'Fco', 140)
df = df.sample(frac=1).reset_index(drop=True)
df


,Id,Sentence,Fco,Pablo,Paula,Vote,Unique,Unique_Vote
0,MODELS 02 EXT 8,"using the lingua franca [28], different host l...",EXTENSION,EXTENSION,EXTENSION,EXTENSION,YES,YES
1,sen_246,Learning/Adaptive Activation Functions: Many c...,LIMITATION,DESCRIPTION,LIMITATION,LIMITATION,NO,NO
2,sen_261,"However, the bottleneck of such methods lies i...",LIMITATION,LIMITATION,LIMITATION,LIMITATION,YES,YES
3,sen_400,Here are some questions about your interaction...,OTHER,OTHER,OTHER,OTHER,YES,YES
4,CHI 02 OVE 19,"advances in ml, largely in language models, ha...",OVERALL,OVERALL,OVERALL,OVERALL,YES,YES
...,...,...,...,...,...,...,...,...
135,sen_61,A large multicenter study demonstrated superio...,RESULT,RESULT,RESULT,RESULT,YES,YES
136,sen_53,The result of the experimental study suggests ...,RESULT,RESULT,RESULT,RESULT,YES,YES
137,ISWC 2022 02 EXT 3,these can be related to the present framework ...,EXTENSION,DESCRIPTION,EXTENSION,EXTENSION,NO,NO
138,sen_183,"Unfortunately, ASAG faces the same difficultie...",LIMITATION,LIMITATION,LIMITATION,LIMITATION,YES,YES


In [ ]:
#df = df[df['Category'] == 'LIMITATION']
#df

In [ ]:
# OUTDATED PROMPT
prompt ="""Text Classification Instructions:\
Objective: Classify excerpts from scientific articles into one of the following seven categories based on their content. Each category corresponds to a specific aspect of scientific discourse, either related to a topic or a study. A topic is defined as a scientific domain, such as “Computer Science” or “Machine  Learning”. A previous study refers to a prior paper on the topic.\

CATEGORIES:\
1. TOPIC - OVERALL: Describes, introduces, classifies, defines or contrasts research topics often based on the discussion of multiple previous studies together (e.g., “Performance estimation strategies can be generally divided into various methods.”).\
2. TOPIC - RESEARCH GAP: Highlights the need for further research within the topic due to absence of information, insufficient information or contradictory information (e.g., “The study of psychopathic traits in digital worlds is under-explored.”).\
3. STUDY - DESCRIPTION: Outlines the objectives, methodology, or design of one previous study, without mentioning results.The outlining of this category is based only in one previous study (e.g., “The author developed an interactive web tool to explore music genres.”).\
4. STUDY - RESULT: Describes specific findings, outcomes, or conclusions drawn from previous studies. This category includes empirical results, theoretical insights, and observed patterns reported by researchers. It often uses verbs like "showed," "found," "demonstrated," or phrases like "the findings indicate" (e.g., “The author found limited effects of nudging on user choices.”).\
5. STUDY - LIMITATION: Describes a constraint, challenge, or weakness inherent in the methodology of a previous study or topic that hinders generalizability or reliability in a particular topic (e.g., “However, their manual creation is labor-intensive and time-consuming, posing obstacles to effective knowledge dissemination.”).\
6. STUDY - EXTENSION: Describes how the current study addresses or extends previous studies by stating the overall idea, contrasting ideas or elaborating further ideas. It usually uses the  words we or our (e.g., “We aim to simplify website design modification through user-described changes.”).\
7. OTHER:  Any text that does not fit the above categories (e.g., “Thanks to Naver AI Lab for their support.”).\

PROCEDURE:\
1. Determine if the text pertains to a topic or a study.\
2. Identify the most suitable category based on the content. Do not create new categories. Use the categories given above.\
3. Provide the category number that best fits the text. Just provide the category number without any explanation.\

Text to Classify:\
"[text]"."""

In [ ]:
# 5. STUDY - LIMITATION: Describes a constraint, challenge, or weakness inherent in the methodology of a previous study or topic that hinders generalizability or reliability in a particular topic or previous study (e.g., “Though these traditional ML methods achieve promising results, they rely heavily on laborious feature engineering.”).\
#5. STUDY - LIMITATION: Describes a constraint, challenge, or weakness inherent in the methodology of a previous study that hinders generalizability or reliability in a particular topic or previous study (e.g., “Though these traditional ML methods achieve promising results, they rely heavily on laborious feature engineering.”).\

#This is the final Prompt
prompt_sonnet_test1 ="""Text Classification Instructions:\
Objective: Classify excerpts from scientific articles into one of the following four categories based on their content. Each category corresponds to a specific aspect of scientific discourse, either related to a topic or a study. A topic is defined as a scientific domain, such as “Computer Science” or “Machine  Learning”. A previous study refers to a prior paper on the topic.\

CATEGORIES:\
1. TOPIC - OVERALL: Describes, introduces, classifies, or defines research topics often based on the discussion of multiple previous studies together (e.g., “Performance estimation strategies can be generally divided into various methods.”).\
2. TOPIC - RESEARCH GAP: Highlights the need for further research within the topic (e.g., “The study of psychopathic traits in digital worlds is under-explored.”).\
3. STUDY - DESCRIPTION: Outlines the objectives, methodology, or design of one previous study, without mentioning results.The outlining of this category is based only in one previous study (e.g., “The author developed an interactive web tool to explore music genres.”).\
4. STUDY - RESULT: Describes specific findings or outcomes drawn from previous studies. This category includes empirical results, theoretical insights, and observed patterns reported by researchers. It often uses verbs like "showed", "found", "demonstrated", "observed" or phrases like "the findings indicate" (e.g., “The author found limited effects of nudging on user choices.”).\
5. STUDY - LIMITATION: Describes a constraint, challenge, or weakness inherent in the methodology of a previous study that hinders generalizability or reliability in a previous study (e.g., “According to Li the key limitation of this task is the lack of standardization.”).\
6. STUDY - EXTENSION: Describes how the current study addresses or extends previous studies by stating the overall idea, contrasting ideas or elaborating further ideas. It usually uses the  words we or our (e.g., “We aim to simplify website design modification through user-described changes.”).\
7. OTHER:  Any text that does not fit the above categories (e.g., “Thanks to Naver AI Lab for their support.”).\

PROCEDURE:\
1. Determine if the text pertains to a topic or a study.\
2. Identify the most suitable category based on the content. Do not create new categories. Use the categories given above.\
3. Provide the category number that best fits the text. Just provide the category number without any explanation.\

Text to Classify:\
"[text]"."""

In [ ]:
prompt=prompt_sonnet_test1


In [ ]:
# Iterate the dataframe
def setPredictedCategory(df,model,bedrock,prompt,column_name):
  for index, row in df.iterrows():
      prompt2 = row["Sentence"]
      final_prompt = prompt.replace("[text]", prompt2)
      response=getPredictedCategory(model,bedrock,final_prompt)
      df.loc[index, column_name] =response
      #print(f"{r}\n\n{final_prompt}")
      print(f"{row['Id']}:  {response}")


In [ ]:
setPredictedCategory(df, "claude-sonnet",bedrock_runtime,prompt,"_Claude-Sonnet 3")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `BedrockChat` was deprecated in LangChain 0.0.34 and will be removed in 0.3. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import ChatBedrock`.
  warn_deprecated(


MODELS 02 EXT 8:  3
sen_246:  2
sen_261:  5
sen_400:  7
CHI 02 OVE 19:  1
sen_219:  3
sen_26:  4
sen_390:  7
sen_94:  5
ISWC 2022 04 LIM 2:  2
sen_40:  3
sen_222:  4
RecSys 02 RES 2:  4
sen_24:  4
RecSys 01 OVE 6:  1
sen_217:  3
NEURIPS04 OVE 2:  6
sen_265:  6
MODELS 02 EXT 10:  3
sen_106:  2
sen_129:  2
sen_315:  5
sen_81:  4
NEURIPS04 OVE 6:  1
NEURIPS04 OVE 4:  1
sen_12:  4
sen_122:  2
sen_254:  1
sen_464:  7
sen_25:  4
sen_33:  4
CHI 01 DES 9:  1
sen_190:  2
RecSys 02 OVE 4:  1
EMNLP03 OVE 2:  1
sen_134:  2
sen_333:  4
sen_234:  6
ISWC 2022 02 OVE 7:  1
sen_223:  6
sen_110:  2
sen_67:  1
sen_127:  2
RecSys 04 OVE 8:  2
RecSys 01 DES 2:  3
sen_186:  2
sen_56:  2
sen_302:  4
CHI 01 DES 3:  3
ISWC 2022 03 OVE 11:  1
NEURIPS04 OVE 7:  1
ISWC 2022 01 EXT 3:  6
ISWC 2022 02 EXT 1:  3
sen_201:  5
RecSys 05 DES 1:  3
sen_465:  7
ISWC 2022 03 DES 1:  3
sen_23:  4
sen_16:  3
sen_120:  2
ISWC 2022 04 OVE 8:  1
sen_436:  7
CHI 02 EXT 9:  6
sen_34:  2
sen_52:  4
EMNLP02 OVE 8:  1
CHI 01 OVE 1: 

In [ ]:
setPredictedCategory(df, "claude-haiku",bedrock_runtime,prompt,"_Claude-haiku")

In [ ]:
setPredictedCategory(df, "llama2-70b",bedrock_runtime,prompt,"_Llama2-70b")

In [ ]:
setPredictedCategory(df, "mistral-7b-instruct",bedrock_runtime,prompt,"_Mistral-7b-instruct")

In [ ]:
setPredictedCategory(df, "mistral-8x7b-instruct",bedrock_runtime,prompt,"_Mistral-8x7b-instruct")

In [ ]:
setPredictedCategory(df, "mistral-large",bedrock_runtime,prompt,"_Mistral-large")

In [ ]:
df

,Id,Sentence,Fco,Pablo,Paula,Vote,Unique,Unique_Vote,_Claude-Sonnet 3
0,MODELS 02 EXT 8,"using the lingua franca [28], different host l...",EXTENSION,EXTENSION,EXTENSION,EXTENSION,YES,YES,3
1,sen_246,Learning/Adaptive Activation Functions: Many c...,LIMITATION,DESCRIPTION,LIMITATION,LIMITATION,NO,NO,2
2,sen_261,"However, the bottleneck of such methods lies i...",LIMITATION,LIMITATION,LIMITATION,LIMITATION,YES,YES,5
3,sen_400,Here are some questions about your interaction...,OTHER,OTHER,OTHER,OTHER,YES,YES,7
4,CHI 02 OVE 19,"advances in ml, largely in language models, ha...",OVERALL,OVERALL,OVERALL,OVERALL,YES,YES,1
...,...,...,...,...,...,...,...,...,...
135,sen_61,A large multicenter study demonstrated superio...,RESULT,RESULT,RESULT,RESULT,YES,YES,4
136,sen_53,The result of the experimental study suggests ...,RESULT,RESULT,RESULT,RESULT,YES,YES,4
137,ISWC 2022 02 EXT 3,these can be related to the present framework ...,EXTENSION,DESCRIPTION,EXTENSION,EXTENSION,NO,NO,6
138,sen_183,"Unfortunately, ASAG faces the same difficultie...",LIMITATION,LIMITATION,LIMITATION,LIMITATION,YES,YES,5


In [ ]:
#df.to_csv('/content/sample_data/Sentence_Classification_Sen140_PSonnet_P_LongChain_v08.csv', index=False)
#df.to_csv('/content/sample_data/Sentence_Classification_Sen350_PSonnet_P_LongChain_v01.csv', index=False)
#df.to_csv('/content/sample_data/Sentence_Classification_Sen700_PSonnet_P_LongChain_v04.csv', index=False)
df.to_csv('/content/sample_data/Sentence_Classification_Raters_PSonnet_P_LongChain_v01.csv', index=False)
#df.to_csv('/content/sample_data/Sentence_Classification_All_PSonnet_P_LongChain_v23.csv', index=False)
#df.to_csv('/content/sample_data/Sen140_PSonnet_Overall_LongChain_v02.csv', index=False)
#df.to_csv('/content/sample_data/Sen140_PSonnet_Limitation_LongChain_v01.csv', index=False)

